## Factors against Parallelism

* Startup costs associated with initiating processes
  * May often overwhelm actual processing time (rendering ||ism useless)
  * Involve thread/process creation, data movement
* Interference: slowdown resulting from multiple processors accessing shared resources
  * Resources: memory, I/O, system bus, sub-processors
  * Software synchronization: locks, latches, mutex, barriers
  * Hardware synchronization: cache faults, interrupts
* Skew: when breaking a single task into many smaller tasks, not all tasks may be the same size
  * Not all tasks finish at the same time
  
### Understanding Factors by Analogy



#### Startup

* Szkieletor in Krakow Poland
  * Too expensive to complete or demolish
* Why is this like a parallel computer?
    * It is a parallel living environment
    * The parallel living throughput is 0 because of startup
    
<img src="./images/sz.png" width="300" title="http://en.wikipedia.org/wiki/File:Szkieletor2.jpg" />

#### Interference: 

* Congested intersections
  * mulitple vehicles compete for same resource (lanes in roundabout)
  * others await resources
* This is a parallel driving environment
  * unused capacity (16 outbound lanes) because resource competition (the roundabout) prevents its use
  * this system exhibits a _throughput collapse_ in which more usage reduces flow
  
<img src="./images/traffic.png" width="512" title="http://crowdcentric.net/2011/05/can-you-help-us-solve-the-worlds-traffic-congestion/" />


#### Skew: 

* Completion of parts for assembly
  * throughput: output planes stalled awaiting other parts
* The parallelism is implicit in the parallel construction of all parts
  * the entire system is stalled (seen) awaiting a nose section.

<img src="./images/plane.png" width="512" title="http://www.ainonline.com/?q=aviation-news/dubai-air-show/2011-11-12/" />


In [ ]:
### In Computer: Real things that Degrade Parllelism

* I/O (memory and storage)
  * may be startup (load data before computation)
  * may be interference (awaiting data transfer between parallel tasks)
  * may be skew (await I/O completion of one task)

* Network communication
    * similar to I/O but always involves communication

* Failures—particularly slow/failed processes (often skew)

The HPC community focuses on communication (among processes) as the major source of slowdown.  This is a traditional (I/O and networking) view.

### Communication

* Parallel computation proceeds in phases
  * Compute (evaluate data that you have locally)
* Communicate (exchange data among compute tasks).  Performance is governed by:
  * Latency: fixed cost to send a message
  * Round trip time (speed of light and switching costs)
* Bandwidth: marginal cost to send a message
  * Link capacity
* Latency dominates small messages and bandwidth dominates large
Almost always better to increase message size for performance, but difficult to achieve in practice.

In [ ]:
### Overlapping Communication and I/O

A simplified model 

In [3]:
import timeit

# synchronous I/O and then compute
def factorial(number):  
    f = 1
    for i in range(2, number+1):
        f *= i
    return f

def io_from_devnull(number):
    with open("/dev/null", "rb") as fh:
        for i in range(number):
            fh.read(1)
            
timeit.timeit(factorial(1000))
timeit.timeit(io_from_devnull(1000))            
timeit.timeit(factorial(1000)  io_from_devnull(1000))

SyntaxError: invalid syntax (<ipython-input-3-b539302310c1>, line 17)

In [2]:
import asyncio  
import time  
from datetime import datetime

async def read_from_devnull():  
    fh = 
async def factorial(name, number):  
    f = 1
    for i in range(2, number+1):
        print('Task {}: Compute factorial({})'.format(name, i))
        await custom_sleep()
        f *= i
    print('Task {}: factorial({}) is {}\n'.format(name, number, f))

start = time.time()  
loop = asyncio.get_event_loop()
tasks = [  
    asyncio.ensure_future(factorial("A", 3)),
    asyncio.ensure_future(factorial("B", 4)),
]
loop.run_until_complete(asyncio.wait(tasks))  
loop.close()
end = time.time()  
print("Total time: {}".format(end - start))

RuntimeError: This event loop is already running

Task A: Compute factorial(2)
SLEEP 2019-01-19 18:21:34.369269
Task A: Compute factorial(3)
SLEEP 2019-01-19 18:21:35.370945
Task A: factorial(3) is 6

Task B: Compute factorial(2)
SLEEP 2019-01-19 18:21:36.372916
Task B: Compute factorial(3)
SLEEP 2019-01-19 18:21:37.373721
Task B: Compute factorial(4)
SLEEP 2019-01-19 18:21:38.375334
Task B: factorial(4) is 24



In [ ]:
### Bulk Synchronous Processing

A simplified model 